In [21]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd

In [22]:
def monumentos():
    monuments = pd.read_csv("../data/monumentos-edificios.csv", delimiter=";")
    monuments['NUM']= monuments['NUM'].astype(str).replace("nan"," SN")
    monuments["Dirección"] = monuments["CLASE-VIAL"] + " " + monuments["NOMBRE-VIA"] + " Nº" + monuments["NUM"]
    return monuments.to_json("../data/monuments.json", orient='records')
def stations():
    stations = pd.read_json("../data/bicimad_stations.json", orient='records')
    geometry_coordinates = stations["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    stations = pd.concat([stations, geometry_coordinates], axis=1)
    return stations.to_json("../data/stations.json", orient='records')

In [23]:
monumentos()
stations()

monuments = pd.read_json("../data/monuments.json", orient='records')
stations = pd.read_json("../data/stations.json", orient='records')

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


In [24]:
stations_clean = stations[["name","address","LONGITUD","LATITUD"]]
stations_clean = stations_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Long_start", "LATITUD": "Lat_start"}, errors="raise")

In [25]:
monuments["Type of place"]= "Edificios de carácter monumental"
monuments_clean = monuments[["NOMBRE","Type of place","Dirección","LONGITUD","LATITUD"]]
monuments_clean = monuments_clean.rename(columns={"NOMBRE": "Place of interest", "Dirección": "Place address", "LONGITUD": "Long_finish", "LATITUD": "Lat_finish"}, errors="raise")

In [26]:
result = pd.merge(monuments_clean,stations_clean, how="cross")

In [27]:
result

,Place of interest,Type of place,Place address,Long_finish,Lat_finish,BiciMAD station,Station location,Long_start,Lat_start
0,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,Puerta del Sol A,Puerta del Sol nº 1,"-3.7018341,",40.417214
1,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,Puerta del Sol B,Puerta del Sol nº 1,"-3.701602938060457,",40.417313
2,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,Miguel Moya,Calle Miguel Moya nº 1,"-3.7058415,",40.420589
3,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"-3.7069171,",40.430294
4,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA Nº3,-3.700924,40.419716,Malasaña,Calle Manuela Malasaña nº 5,"-3.7025875,",40.428552
...,...,...,...,...,...,...,...,...,...
55963,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"-3.72699,",40.443750
55964,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"-3.72693,",40.443420
55965,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,Facultad Biología,Calle José Antonio Novais frente al nº 12,"-3.7272945,",40.448332
55966,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO Nº SN,-3.761360,40.411760,Facultad Derecho,Avenida Complutense nº 23,"-3.72937,",40.451090


In [ ]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c